In [1]:
import json

with open('data.json', 'r') as f:
    data_dict_list = json.load(f)

In [2]:
import pandas as pd
import re
from rusenttokenize import ru_sent_tokenize
import itertools
from pymorphy2 import MorphAnalyzer
import collections
from collections import Counter, defaultdict
import numpy as np
from string import punctuation
from nltk.corpus import stopwords
morph = MorphAnalyzer()

In [3]:
import glob
import os

path = 'Сематические Алгоритмы_texted/'

sentences_dict = {}

for filename in glob.glob(os.path.join(path, '*.txt')):    
    with open(filename, 'r') as f:
        text = f.read()
        text = text.replace('\n', ' ')
        text = re.sub(' +', ' ', text)
        sentences = ru_sent_tokenize(text)
        sentences_dict[os.path.basename(filename)[:-4]] = sentences

In [5]:
for data_dict in data_dict_list:
    if data_dict['id'] in sentences_dict:
        print(sentences_dict[data_dict['id']])
        for items in data_dict['markups']:
            for item in items['escape']:
                print(item['text'])
                print(item['tags'])
        print("\n")

['тел.:8-800-250-60-70 (звонок по России бесплатный) СПЕЦСВЯЗЬ № накладной | 409901654951 экспресс Федеральное государственное унитарное предприятие Дата "Главный центр специальной связи" заполнения 2019-06-19 www.cccb.ru Заполняется отправителем ( 5-слойный бланк, пишите с нажимом) Договор № ee _ 3.', 'ОПИСАНИЕ ВЛОЖИМОГО ш Индекс вложения Документация 4.', 'ОСОБЫЕ ОТМЕТКИ 5.', 'ОЦЕНОЧНАЯ CTOUMOCTb Я подтверждаю, что отправление не содержит запрещенные к пересылке предметы и вещества.', 'С правилами пересылки ознакомлен.', 'Согласен на обработку персональных данных.', 'тел.:8-800-250-60-70 (звонок по России бесплатный) ЭКСПРЕСС [ ] БИЗНЕС Заполняется сотрудником ФГУП ГЦСС 6.', 'ВИД ПРИЕМА Ш По вызову Ш В окне приема 7.', 'УПАКОВКА (пломбы) 8.', 'ГАБАРИТЫ И МАССА 9.СВЕДЕНИЯ ОБ ОПЛАТЕ Взята наложенная плата, руб.', '[ Оплата наличными |:| 10.ПРИЕМ ОТПРАВЛЕНИЙ _ press [_iommcsrce Время 11.ИНФОРМАЦИЯ О ВРУЧЕНИИ ОТПРАВЛЕНИЯ Способ вручения: LJ NUYHO 0 через доверенное лицо Служебные отметки

In [10]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [54]:
dogovor = pd.read_csv('bagofwords.csv')
dogovor.head()

,text,label
0,СТАТЬЯ 3. ДОГОВОРНАЯ ЦЕНА ( п.п. 3.1 3.20),1
1,3.1. Договорная цена формируется по Договору в...,1
2,3.1.1 По мере выхода рабочей документации форм...,1
3,Стоимость Договора/Задания на строительство яв...,1
4,Стоимость Договора включает в себя:,1


In [84]:
all_sentences = list()
for punkt in dogovor['text'].tolist():
    all_sentences += ru_sent_tokenize(punkt)

In [85]:
stops = set(stopwords.words('russian'))
def normalize(text):
    normalized_text = [morph.parse(word.strip(punctuation))[0].normal_form for word \
                                                            in text.lower().split()]
    normalized_text = [word for word in normalized_text if word not in stops]
    return normalized_text

In [86]:
# all_sentences = [['<start>'] + normalize(sentence) + ['<end>'] for sentence in all_sentences]
all_sentences = [normalize(sentence) for sentence in all_sentences]

In [87]:
all_sentences

[['статья', '3'],
 ['договорный', 'цена', '', 'п.п', '3.1', '3.20'],
 ['3.1',
  'договорный',
  'цена',
  'формироваться',
  'договор',
  'соответствие',
  'приложение',
  '№5',
  'составлять'],
 ['3.1.1',
  'мера',
  'выход',
  'рабочий',
  'документация',
  'формироваться',
  'наряд-заказ'],
 ['стоимость',
  'договора/задание',
  'строительство',
  'являться',
  'ориентировочный',
  'состоять',
  'стоимость',
  'наряд-заказ',
  'подлежать',
  'корректировка',
  'посредством',
  'внесение',
  'изменение',
  'посредством',
  'заключение',
  'дополнительный',
  'соглашение',
  'договор'],
 ['стоимость', 'договор', 'включать'],
 ['стоимость', 'работа', 'наряд-заказ'],
 ['стоимость', 'поставка', 'оборудование', 'материал', 'подрядчик'],
 ['резерв',
  'средство',
  'заказчик',
  'предназначить',
  'возмещение',
  'работа',
  'затрата',
  'потребность',
  'который',
  'возникать',
  'ход',
  'строительство',
  'результат',
  'уточнение',
  'проектный',
  'решение',
  'условие',
  'строитель

In [14]:
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
        
    return ngrams

In [15]:
unigrams = Counter()
bigrams = Counter()

for sentence in all_sentences:
    unigrams.update(sentence)
    bigrams.update(ngrammer(sentence))

In [16]:
bigrams.most_common(50)

[('выполнение работа', 68),
 ('настоящий договор', 49),
 ('материал оборудование', 40),
 (' п.п', 30),
 ('выполнить работа', 26),
 ('заказчик вправе', 23),
 ('рабочий документация', 19),
 ('штрафной санкция', 19),
 ('задание строительство', 18),
 ('срок выполнение', 18),
 ('акт приёмка', 17),
 ('передать подрядчик', 17),
 ('строительство объект', 15),
 ('область пэб', 14),
 ('пэб гз', 14),
 ('закончить строительство', 14),
 ('исполнение договор', 13),
 ('договор заказчик', 12),
 ('подписание акт', 12),
 ('третье лицо', 12),
 ('приёмка закончить', 12),
 ('вид работа', 11),
 ('график выполнение', 11),
 ('форма приложение', 11),
 ('производство работа', 11),
 ('ключевой событие', 11),
 ('электронный носитель', 11),
 ('приложение №8', 11),
 ('случай нарушение', 11),
 ('подрядчик обязать', 11),
 ('нарушение штрафной', 11),
 ('область пб', 11),
 ('стоимость работа', 10),
 ('работа объект', 10),
 ('строительный контроль', 10),
 ('подрядчик электронный', 10),
 ('форма заказчика»', 10),
 ('усло

In [17]:
unigrams.most_common(50)

[('подрядчик', 263),
 ('заказчик', 211),
 ('работа', 198),
 ('договор', 174),
 ('выполнение', 102),
 ('приложение', 97),
 ('случай', 76),
 ('срок', 75),
 ('сторона', 70),
 ('наряд-заказ', 63),
 ('акт', 60),
 ('', 58),
 ('материал', 58),
 ('объект', 54),
 ('форма', 54),
 ('нарушение', 54),
 ('настоящий', 52),
 ('требование', 48),
 ('строительство', 47),
 ('оборудование', 47),
 ('исполнение', 43),
 ('стоимость', 40),
 ('статья', 39),
 ('день', 38),
 ('выполнить', 37),
 ('область', 35),
 ('соответствие', 34),
 ('изменение', 34),
 ('условие', 34),
 ('документация', 33),
 ('п.п', 30),
 ('указанный', 30),
 ('соответствующий', 30),
 ('подписание', 29),
 ('и/ить', 29),
 ('обязательство', 27),
 ('вправе', 27),
 ('течение', 26),
 ('документ', 26),
 ('пункт', 25),
 ('предусмотренный', 25),
 ('рабочий', 24),
 ('дата', 24),
 ('это', 24),
 ('порядок', 24),
 ('аванс', 24),
 ('приёмка', 24),
 ('свой', 24),
 ('штрафной', 24),
 ('гарантия', 24)]

In [18]:
def collect_stats(texts, stops):
    
    unigrams = Counter()
    bigrams = Counter()
    
    for text in texts:
        unigrams.update(text)
        bigrams.update(ngrammer(text, 2))
    
    return unigrams, bigrams

In [19]:
def scorer(word_count_a, word_count_b, bigram_count, len_vocab, min_count):
    try:
        score = ((bigram_count - min_count) / (word_count_a * word_count_b)) * len_vocab  
        # ((word_count_a + word_count_b)))
    except ZeroDivisionError:
        return 0
    
    return score

In [20]:
def score_bigrams(unigrams, bigrams, scorer, threshold=-10, min_count=2):
    bigram2score = Counter()
    len_vocab = len(unigrams)
    for bigram in bigrams:
        if len(bigram.split())>1:
            score = scorer(unigrams[bigram.split()[0]], unigrams[bigram.split()[1]], 
                       bigrams[bigram], len_vocab, min_count)
        
        if score > threshold:
            bigram2score[bigram] = score
    
    return bigram2score

In [21]:
unigrams, bigrams = collect_stats(all_sentences, stops)

In [22]:
bigram2score = score_bigrams(unigrams, bigrams, scorer)

In [23]:
bigram2score.most_common(50)

[('оценочный лист', 171.48),
 ('№14 «мотивация', 158.77777777777777),
 ('3-го уровень', 119.08333333333333),
 ('охрана труд', 119.08333333333333),
 ('безусловный безотзывный', 119.08333333333333),
 ('сто тысяча', 119.08333333333333),
 ('8.1.1 «соглашение', 119.08333333333333),
 ('принятие решение', 102.07142857142857),
 ('законодательство рф', 102.07142857142857),
 ('ключевой событие', 97.43181818181817),
 ('лист деятельность', 95.26666666666667),
 ('финансирование погашение', 95.26666666666667),
 ('случайный гибель', 95.26666666666667),
 ('10 десять', 89.3125),
 ('электронный носитель', 82.4423076923077),
 ('мера выход', 79.38888888888889),
 ('показатель эффективность', 79.38888888888889),
 ('подрядный организация', 77.94545454545454),
 ('договорный цена', 75.78030303030303),
 ('цена ', 75.78030303030303),
 (' п.п', 75.78030303030303),
 ('первичный учётный', 71.45),
 ('оформлять безусловный', 71.45),
 ('план управление', 71.45),
 ('перечисление денежный', 68.04761904761904),
 ('лиценз

In [24]:
def get_window_stats(texts, window=3):
    
    bigrams = defaultdict(list)

    for text in texts:
        for i in range(len(text)-window):
            words = list(enumerate(text[i:i+window]))
            target = words[0][1]
            for j, word in words[1:]:
                bigrams[(target, word)].append(j)
    
    bigrams_stds = Counter()
    for bigram in bigrams:
        if len(bigrams[bigram]) > 3:
            bigrams_stds[bigram] = np.std(bigrams[bigram])
    
    return bigrams_stds

In [25]:
bigram2std = get_window_stats(all_sentences)

In [26]:
bigram2std.most_common()[:-20:-1]

[(('исполнение', 'наряд-заказ'), 0.0),
 (('форс-мажорный', 'обстоятельство'), 0.0),
 (('пб»', '№13'), 0.0),
 (('пб»', 'приложение'), 0.0),
 (('область', 'приложение'), 0.0),
 (('область', 'пб»'), 0.0),
 (('«соглашение', 'пб»'), 0.0),
 (('8.1.1', 'область'), 0.0),
 (('8.1.1', '«соглашение'), 0.0),
 (('плата', '«соглашение'), 0.0),
 (('плата', '8.1.1'), 0.0),
 (('условие', '8.1.1'), 0.0),
 (('условие', 'плата'), 0.0),
 (('заказчик', 'размер'), 0.0),
 (('уплачивать', 'заказчик'), 0.0),
 (('подрядчик', 'уплачивать'), 0.0),
 (('подрядчик', 'установленный'), 0.0),
 (('потребовать', 'подрядчик'), 0.0),
 (('вправе', 'потребовать'), 0.0)]

In [59]:
#bigram2std.keys()

In [60]:
import glob
import os
path = 'Сематические Алгоритмы_texted/'#'./data_filtered/'
for filename in glob.glob(os.path.join(path, '*.txt')):
    with open(filename, 'r') as f:
        text = f.read()
        text = text.replace('\n', ' ')
        text = re.sub(' +', ' ', text)
        sentences = ru_sent_tokenize(text)
        print(sentences)
        print()
        sentences = [normalize(sentence) for sentence in sentences]
        bigrams_tgt = Counter()
        # bigrams_tgt = get_window_stats(sentences)
        for sentence in sentences:
            bigrams_tgt.update(ngrammer(sentence))
        # print(bigrams_tgt)
        print('extrcated key phrases:')
        for key in bigrams_tgt.keys():
            if key in bigrams.keys():
                print(key)
        print("="*80)
        print()

["rasnponОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ«ГАЗПРОМНЕФТЬ-ЯМАЛ' АЭПРОМНЕЙ о »(000 «ГА: еФТЬ-ЯМАЛ») Руководителю проекта «УКПГ»Юрндический wiped: АО «Трест Коксохиммонтаж»Россия, 629002 Салехард, уп.", 'Мира, д. 43АOFPH 1118903004989, WHH 8901001822 n.A. 6oS Pa ere = ыы ‚„А. БеляевАдрес для корреспонденции: уРоссия, 625048, Тюмень, ул. 50 лет Октября, л, 85Ten.', '+7 (3452) 52-10-90 Факс: +7 (3452) 52--91-64e-mail gon-yamai@tmn,gazorom—neft ruwww.yamal gazprom—nelt.reLELC. LLL.', 'LHOT E32Ha Neo =aО согласовании СубподрядчиковУважаемый Петр Алексеевич!Рассмотрев направленную Вами информацию в отношении:- OOO «СПЕКТР-ЭЛЕКТРО», UHH 2320154860, оказание услуг строительнойлабораторией (испытание свай);- ООО «СК Альфа Регион», ИНН 5406574870, строительно-монтажные работы;- ООО «АСУ ПРО», ИНН 5610083053, пуско-наладочные работы;- ООО «АС Инжстрой», ИНН 7728845775, строительно-монтажные работы;- ОАО «ДМНР-КХМ», ИНН 7722104548, Строительно-монтажные работы, Пуско-наладочные работы- ООО «Ко

extrcated key phrases:
 
подготовка газа
газа конденсат
компримирование подготовка
объект «комплекс
«комплекс объект
объект компримирование
ооо «газпромнефть-ямал»
строительный норма
норма правило
геодезический съёмка
ао «трест
дальнейший использование

['@ глзпРОМ ‚ ° ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ «ГАЗПРОМНЕФТЬ-РАЗВИТИЕ » Заместителю генерального директора 002 «гПн-2АЗИТИ АО "ТРЕСТ КОКСОХИММОНТАЖ" ПлРОСКРР ад ” it.', 'Сонк-Пеербул Зна paecaee wep a 2-93 А.Л. Зажигину lon , ° _.', 'r з » he $ ._‘: .', 'WN ep ase i tt !', 'LE LO.LAP» LUGEPO aN с me нн Об аудите медицинских организаций Уважаемый Андрей Леонидович!', 'Уведомляем Вас, что 10.10.2019 был проведен аудит медицинских организаций: ГБУЗ ° ЯНАО — «Надымская UPB» — (Ямало-Ненецкий — автономный — округ г. Надым, ул. Сенькина, дом 2) и ООО «Ямальская Трассовая Медсанчасть» (Ямало-Ненецкий автономный округ, г. Надым, поселок Финский комплекс, д.19).', 'По результатам проведенного аудита, медицинские организации соответству

extrcated key phrases:
выполнение строительно-монтажный
строительно-монтажный работа
работа объект
объект «комплекс
«комплекс объект
компримирование подготовка
подготовка газа
газа конденсат
выполнение работа
работа соответствие
соответствие условие
условие договор
технический людская
людская ресурс
требование область
область пэб
пэб гз
транспортный средство
объект заказчик
обязательство рамка
договорный отношение
договор сторона
сторона заказчик
графика выполнение
начало работа
банковский гарантия
оценка эффективность
промышленный безопасность
требование пб
производство работа
план управление
управление договор
подрядчик субподрядчик
персонал работа
соблюдение требование
требование производственный
штрафной санкция
экологический безопасность
контроль качество
ао «трест
«трест коксохиммонтаж»
срок выполнение
строительный контроль
назначить ответственный
объект 

['@ гАЗПРОМ НЕФ ТЬ ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ Заместителю руководителя проекта «ГАЗПРОМНЕФТЬ-РАЗВИТИЕ» АО «ТРЕС

extrcated key phrases:
подрядный организация
транспортный средство
выполнение работа
работа соответствие
соответствие условие
подписание договор
сторона заказчик
производственный безопасность
основной правило
требование область
область пэб
пэб гз
ооо «газпромнефть-ямал»
производство работа
работа порядок
безопасность охрана
охрана труд
работа объект
строительно-монтажный работа
работа также
исполнительный документация
объект заказчик
строительный контроль
контроль соблюдение
проектный решение
ооо «гпн-развитие»
право обязанность
получить заказчик
заказчик информация
 
ао «трест
необходимый документ
условие договор
документация 
контроль качество
работа подрядчик
«трест коксохиммонтаж»
дополнительный соглашение

['безопасности, повлекшие возникновение пожара и причинение тяжкого вреда здоровью или смерть человека, инциденты аварии на объектах энергохозяйства (B T.4. повреждение коммуникаций), нарушения — требований — антиалкогольной/антинаркотической — политики, разглашение конфиденциал

extrcated key phrases:
«трест коксохиммонтаж»
выполнение работа
объект «комплекс
«комплекс объект
объект компримирование
компримирование подготовка
подготовка газа
газа конденсат
новопортовский нефтегазоконденсатный
ао «трест

['ООО «Трест КХМ-Бурение» 7705936440ООО «Строительство- Электромонтажные раб7 ктро работы[1 | ©®а ——| оо19 O00 «KXM-C3» 4704039890 Строительно — монтажные работы1 2 000 “ eee 7705800601 Строительно — монтажные работыонтажник»OOO HITO«ФундаментСтройАркос» 7203106421 Термостабилизация грунтовООО «Объединенная сервисная 7445033042 Обслуживание городка (клининг)компания»П ООО «Сервисно- 9709039815 Организация питанияродовольственная компания» ЧОО «Витязь» 8907002371Охранная деятельность Приложение 1: письмо №14-05/7881 от 16.17.2019г — 1 лист.Приложение 2: план-график мобилизации — | файл.ВРуководитель проектаАО «Трест Коксохиммонтаж» П.А. БеляевАкционерное общество ) Stock companyТРЕСТ КОКСОХИММОНТАЖ М TRUST KOKSOKHIMMONTAZHСистема качества соответствует международн

extrcated key phrases:
объём работа
 
объект «комплекс
«комплекс объект
объект компримирование
компримирование подготовка
подготовка газа
газа конденсат
ооо «газпромнефть-ямал»
строительный норма
норма правило
полный объём
ао «трест
дальнейший использование

['@ гзпРОМTe aeОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ\'5‘3”"0\'“_\'.‘{.‘„”_7?‚”.\'.“."”\';... ы Генеральному директору- й AO «TPECT KXM»ЗАМЕСТИТЕЛЬ ГЕНЕРАЛЬНОГО ДИРЕКТОРАпридический адре‹Россия, 629002, Салехард, ул. Мира, д. 4ЗА C.B. ®ycbaesyOFPH 1118903004989, UHH 8901001822‘\\дуб я корреспонденцииюмень, ул 50 \' RIWODH, 2192) 52-10-90.', 'Maxc: +7 (3452) 52-91-6416 072019 ,, - 05 [ 1881OyО взаимодействииУважаемый Сергей Валентинович!В рамках Договора № ГНЯ-19/11024/00865/Р/9-03 от 05.06.2019 на выполнение работ построительству объекта: «Комплекса объектов по компримированию и подготовке газа иконденсата (УКПГ) Новопортовского нефтегазоконденсатного месторождения» дополнительнок электронным письмам были официально направлены в 

extrcated key phrases:
ао «трест
договор №
выполнение строительно-монтажный
строительно-монтажный работа
«трест коксохиммонтаж»
согласование субподрядчик

['Акционерное общество № Stock Company TPECT RORCOXUMMONTAM TRUST KOKSOKHIMMONTAZH РФ, г. Москва, 115035 TOM (495) 953 22 68 kxm@kxm.ru tel.: +7495 9532 Кадашевская наб., 36/5 факс: (495) 953 59 75 Wwww.kxm.ru fax: +7495 9535 68 36/5 Kadashevskaya nab., 75 2 9 115035, Moscow, Russia Приложение: |.График отпуска воды для хозяйственно-бытовых и технологических нужд АО «’Тест КХМ» на лО1О С уважением, Заместитель Руководителя проекта ЦИС АО «Трест Коксохиммонтаж» Д.С. НовосёловdOlgoodoH -)` `отеПжаэ91, , : !', '0520 # nNBboOOW d OJOLY ¢*0 IS >KAH QUNDSh изоцонхе..', '-OHH@ESLO Togenuod| | cWOCOL ЯОжЖАнН эчвощяо «INX) 122 ]» е / 006 © NBOOW @ OJOLY eNO I9>KAH SUNOS VUJOLIOHXOL -OHHOSLO Togeuod| | ¢ W006 € ladKAY 31901199 CW 050 7 NKBOSW @ OJOLY е WOEL и /ЛО @ OJOLY 6^ 0 ЯОАН OUNDSh UJOLIOHXEL -OHHOELO Vogeuod|]} с ^ 050 # ЯОЖАН эчволяо с

extrcated key phrases:
 
планировать заключение
производственный безопасность
соответствующий требование
пао «газпром
«газпром нефть»



## По классам

In [96]:
class_map = {0: "другое", 1: "стоимость", 2: "сроки", 3: "качество"}

In [97]:
punkts_other = dogovor[dogovor.label==0]
punkts_other = punkts_other['text'].tolist()

In [98]:
punkts_other

['Рамочный Договор № ГНЯ-19/11024/00865/Р/9-03/ГНР-19/11024/00592/Р/14 от 05.06.2019 с АО «ТРЕСТ КОКСОХИММОНТАЖ» на выполнение строительно-монтажных работ по объекту «Комплекс объектов по компримированию и подготовке газа и конденсата (УКПГ) Новопортовского нефтегазоконденсатного месторождения»',
 'СТАТЬЯ 2. Предмет договора ( п.п. 2.1 2.8)',
 '2.8. В целях своевременного выполнения Работ по настоящему Договору, Заказчик предоставляет документацию согласно п.9.3 настоящего Договора.',
 'РАЗДЕЛ II. ОБЩИЕ ОБЯЗАТЕЛЬСТВА СТОРОН',
 'СТАТЬЯ 7. ЗАВЕРЕНИЯ ОБ ОБСТОЯТЕЛЬСТВАХ, ИМЕЮЩИХ ЗНАЧЕНИЕ ДЛЯ ЗАКЛЮЧЕНИЯ, ИСПОЛНЕНИЯ ИЛИ ПРЕКРАЩЕНИЯ ДОГОВОРА ( п.п. 7.1 7.9)',
 '7.1.7. Подрядчик, подписывая Перечень нарушений и штрафных санкций (Приложение №9 к Договору), ознакомлен с размерами штрафных санкций, подтверждает их соразмерность последствиям возможных нарушений. В случае наличия в условиях Договора и любых приложениях к нему противоречий о порядке/форме фиксации и размере штрафных санкций по тожде

In [63]:
punkts_cost = dogovor[dogovor.label==1]
punkts_cost = punkts_cost['text'].tolist()

In [99]:
punkts_cost

['СТАТЬЯ 3. ДОГОВОРНАЯ ЦЕНА ( п.п. 3.1 3.20)',
 '3.1. Договорная цена формируется по Договору в соответствии с Приложением №5 и составляет:',
 '3.1.1 По мере выхода рабочей документации формируются Наряд-заказы.',
 'Стоимость Договора/Задания на строительство является ориентировочной, состоит из стоимости Наряд-заказов и подлежит корректировке посредством внесения изменений посредством заключения Дополнительного соглашения к Договору.',
 'Стоимость Договора включает в себя:',
 'стоимость Работ по Наряд-Заказам;',
 'стоимость поставки Оборудования и Материалов Подрядчика;',
 'резерв средств Заказчика, предназначенного для возмещения работ и затрат, потребность в которых возникает в ходе строительства в результате уточнения проектных решений и условий строительства;',
 'другие затраты Подрядчика, определяемые Сторонами в соответствии с Приложением №6.1',
 'ставку за досрочное завершение комплекса строительно-монтажных работ, указанных в Задании на строительство в размере не более 5% от с

In [101]:
punkts_terms = dogovor[dogovor.label==2]
punkts_terms = punkts_terms['text'].tolist()

In [102]:
punkts_quality = dogovor[dogovor.label==3]
punkts_quality = punkts_quality['text'].tolist()

In [100]:
sentences_other = list()
for punkt in punkts_other:
    sentences_other += ru_sent_tokenize(punkt)

In [103]:
punkts_cost

['СТАТЬЯ 3. ДОГОВОРНАЯ ЦЕНА ( п.п. 3.1 3.20)',
 '3.1. Договорная цена формируется по Договору в соответствии с Приложением №5 и составляет:',
 '3.1.1 По мере выхода рабочей документации формируются Наряд-заказы.',
 'Стоимость Договора/Задания на строительство является ориентировочной, состоит из стоимости Наряд-заказов и подлежит корректировке посредством внесения изменений посредством заключения Дополнительного соглашения к Договору.',
 'Стоимость Договора включает в себя:',
 'стоимость Работ по Наряд-Заказам;',
 'стоимость поставки Оборудования и Материалов Подрядчика;',
 'резерв средств Заказчика, предназначенного для возмещения работ и затрат, потребность в которых возникает в ходе строительства в результате уточнения проектных решений и условий строительства;',
 'другие затраты Подрядчика, определяемые Сторонами в соответствии с Приложением №6.1',
 'ставку за досрочное завершение комплекса строительно-монтажных работ, указанных в Задании на строительство в размере не более 5% от с

In [104]:
sentences_cost = list()
for punkt in punkts_cost:
    sentences_cost += ru_sent_tokenize(punkt)

In [105]:
sentences_terms = list()
for punkt in punkts_terms:
    sentences_terms += ru_sent_tokenize(punkt)

In [106]:
sentences_quality = list()
for punkt in punkts_quality:
    sentences_quality += ru_sent_tokenize(punkt)

In [107]:
sentences_other = [normalize(sentence) for sentence in sentences_other]
sentences_cost = [normalize(sentence) for sentence in sentences_cost]
sentences_terms = [normalize(sentence) for sentence in sentences_terms]
sentences_quality = [normalize(sentence) for sentence in sentences_quality]

In [108]:
unigrams_other = Counter()
bigrams_other = Counter()

for sentence in sentences_other:
    unigrams_other.update(sentence)
    bigrams_other.update(ngrammer(sentence))

In [109]:
unigrams_cost = Counter()
bigrams_cost = Counter()

for sentence in sentences_cost:
    unigrams_cost.update(sentence)
    bigrams_cost.update(ngrammer(sentence))

In [110]:
unigrams_terms = Counter()
bigrams_terms = Counter()

for sentence in sentences_terms:
    unigrams_terms.update(sentence)
    bigrams_terms.update(ngrammer(sentence))

In [111]:
unigrams_quality = Counter()
bigrams_quality = Counter()

for sentence in sentences_quality:
    unigrams_quality.update(sentence)
    bigrams_quality.update(ngrammer(sentence))

In [112]:
print(unigrams_other.most_common(15))
print('-'*90)
print()
print(unigrams_cost.most_common(15))
print('-'*90)
print()
print(unigrams_terms.most_common(15))
print('-'*90)
print()
print(unigrams_quality.most_common(15))

[('подрядчик', 189), ('заказчик', 152), ('договор', 126), ('работа', 105), ('случай', 62), ('выполнение', 56), ('приложение', 53), ('нарушение', 52), ('объект', 49), ('материал', 46), ('срок', 46), ('сторона', 45), ('требование', 41), ('настоящий', 40), ('акт', 40)]
------------------------------------------------------------------------------------------

[('работа', 41), ('подрядчик', 36), ('заказчик', 32), ('договор', 30), ('приложение', 24), ('стоимость', 18), ('наряд-заказ', 16), ('изменение', 16), ('средство', 14), ('выполнение', 14), ('аванс', 14), ('сторона', 13), ('рабочий', 12), ('документация', 11), ('строительство', 11)]
------------------------------------------------------------------------------------------

[('работа', 45), ('выполнение', 30), ('подрядчик', 27), ('форма', 23), ('приложение', 20), ('заказчик', 19), ('срок', 18), ('наряд-заказ', 14), ('договор', 12), ('выполнить', 11), ('', 10), ('передать', 10), ('сторона', 10), ('акт', 9), ('графика', 9)]
--------------

In [113]:
print(bigrams_other.most_common(15))
print('-'*90)
print()
print(bigrams_cost.most_common(15))
print('-'*90)
print()
print(bigrams_terms.most_common(15))
print('-'*90)
print()
print(bigrams_quality.most_common(15))

[('выполнение работа', 38), ('настоящий договор', 38), ('материал оборудование', 34), (' п.п', 24), ('штрафной санкция', 18), ('заказчик вправе', 17), ('закончить строительство', 14), ('строительство объект', 14), ('исполнение договор', 13), ('пэб гз', 13), ('область пэб', 13), ('третье лицо', 12), ('приёмка закончить', 12), ('нарушение штрафной', 11), ('акт приёмка', 11)]
------------------------------------------------------------------------------------------

[('рабочий документация', 11), ('авансовый средство', 9), ('выполнение работа', 8), ('договорный цена', 6), ('вид работа', 6), ('выполнить работа', 6), ('соответствие приложение', 5), ('дополнительный соглашение', 5), ('стоимость работа', 5), ('задание строительство', 5), ('погашение аванс', 5), ('использование авансовый', 5), ('нецелевой использование', 5), ('работа наряд-заказ', 4), ('выплата подрядчик', 4)]
------------------------------------------------------------------------------------------

[('выполнение работа', 20)

ДРУГОЕ
ао «трест
договор №
«трест коксохиммонтаж»
согласование субподрядчик
выполнение строительно-монтажный
--------------------------------------------------------------------------------
СТОИМОСТЬ
строительно-монтажный работа
--------------------------------------------------------------------------------
СРОКИ
работа 
--------------------------------------------------------------------------------

ДРУГОЕ
«трест коксохиммонтаж»
«комплекс объект
объект компримирование
компримирование подготовка
подготовка газа
газа конденсат
конденсат укпг
укпг новопортовский
договор №
ао «трест
письменный согласие
увеличение срок
--------------------------------------------------------------------------------
СРОКИ
продление срок
--------------------------------------------------------------------------------
КАЧЕСТВО
контроль соблюдение
--------------------------------------------------------------------------------
СТОИМОСТЬ
соблюдение требование
--------------------------------------------------

ДРУГОЕ
«трест коксохиммонтаж»
объект «комплекс
«комплекс объект
объект компримирование
компримирование подготовка
подготовка газа
газа конденсат
новопортовский нефтегазоконденсатный
ао «трест
--------------------------------------------------------------------------------
СТОИМОСТЬ
выполнение работа
--------------------------------------------------------------------------------

ДРУГОЕ
«трест коксохиммонтаж»
ао «трест
--------------------------------------------------------------------------------
СРОКИ
технический людская
людская ресурс
работа 
--------------------------------------------------------------------------------
СТОИМОСТЬ
выполнять работа
 
--------------------------------------------------------------------------------

СТОИМОСТЬ
объём работа
 
--------------------------------------------------------------------------------
ДРУГОЕ
объект «комплекс
«комплекс объект
объект компримирование
компримирование подготовка
подготовка газа
газа конденсат
ооо «газпромнефть-ямал»
пол

## TFIDF

In [78]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [88]:
all_sentences = [' '.join(sent) for sent in all_sentences]

In [89]:
all_sentences

['статья 3',
 'договорный цена  п.п 3.1 3.20',
 '3.1 договорный цена формироваться договор соответствие приложение №5 составлять',
 '3.1.1 мера выход рабочий документация формироваться наряд-заказ',
 'стоимость договора/задание строительство являться ориентировочный состоять стоимость наряд-заказ подлежать корректировка посредством внесение изменение посредством заключение дополнительный соглашение договор',
 'стоимость договор включать',
 'стоимость работа наряд-заказ',
 'стоимость поставка оборудование материал подрядчик',
 'резерв средство заказчик предназначить возмещение работа затрата потребность который возникать ход строительство результат уточнение проектный решение условие строительство',
 'затрата подрядчик определять сторона соответствие приложение №6.1',
 'ставка досрочный завершение комплекс строительно-монтажный работа указанный задание строительство размер 5 стоимость работа дать ставка подлежать выплата подрядчик случай принятие решение соответствие приложение №14 «мот

In [90]:
tfidf = TfidfVectorizer(ngram_range=(1,2), max_df=0.5)

In [91]:
tfidf.fit(all_sentences)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.5, max_features=None,
                min_df=1, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [92]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}

In [93]:
texts_vectors = tfidf.transform(all_sentences)

In [94]:
keywords = []

for row in range(texts_vectors.shape[0]):
    row_data = texts_vectors.getrow(row)
    top_inds = row_data.toarray().argsort()[0,:-11:-1]
    keywords.append([id2word[w] for w in top_inds])

In [95]:
for pair in zip(all_sentences, keywords):
    print(pair[0])
    print()
    print(pair[1])
    print("="*80)

статья 3

['статья', 'ямал огрн', 'который', 'копия договор', 'корпоративный', 'корпоративный защита', 'корректировать', 'корректировать мероприятие', 'корректировка', 'корректировка посредством']
договорный цена  п.п 3.1 3.20

['цена 20', 'договорный цена', 'цена', 'договорный', '20', 'который возможно', 'корректировать', 'корректировать мероприятие', 'корректировка', 'корректировка посредством']
3.1 договорный цена формироваться договор соответствие приложение №5 составлять

['цена формироваться', 'приложение составлять', 'формироваться договор', 'формироваться', 'договор соответствие', 'составлять', 'соответствие приложение', 'договорный цена', 'цена', 'договорный']
3.1.1 мера выход рабочий документация формироваться наряд-заказ

['документация формироваться', 'формироваться наряд', 'мера выход', 'выход', 'выход рабочий', 'формироваться', 'мера', 'рабочий документация', 'рабочий', 'документация']
стоимость договора/задание строительство являться ориентировочный состоять стоимость на